In [117]:
import cv2
import pykep
import numpy as np
import imutils
import math

Image tools

In [208]:
def debris_in_pixels(physical_size, distance, alpha=np.pi/6, beta=np.pi/6, dimensions = (1080, 1980)):
    """
    Returns how many pixels the object at a given distance should take up
    :param physical_size: The size of the object
    :param distance: Distance between the target and observer
    :param alpha: horizontal FOV. Radians
    :param beta: vertical FOV. Radians
    :param dimensions: the pixel dimensions of the observer camera used
    """
    print(distance)
    len_x = 2*np.tan(alpha)*distance
    len_y = 2*np.tan(beta)*distance
    print("lengths", len_x,len_y)
    px_per_length_cols =  dimensions[0]/len_x
    px_per_length_rows =  dimensions[1]/len_y
    new_shape = math.ceil(physical_size[0] * px_per_length_cols), math.ceil(physical_size[1] * px_per_length_rows)
    # think about the rounding here
    return new_shape


def resize(original_image, physical_size, distance):
    """
    Resize the image based on the physical size of the target object and the relative distance between 
    target object and observer
    :param physical_size: The size of the object in km
    :param distance: Distance between the target and observer planes, the z coordinate of the relative vector
    """
    width, height = debris_in_pixels(physical_size, distance)
    if width == 0 and height ==0:
        return False  # can't resize, object too small
    else:
        return cv2.resize(original_image, (width, height), interpolation=cv2.INTER_AREA)

In [206]:
debris_in_pixels((10,10), 5000)

5000
lengths 5773.502691896258 5773.502691896258


(1, 3)

In [211]:
def roi_in_frame(left_corner, object_size, frame):
    """
    Return the part of the ROI that is in the frame.
    :left_corner: the coordinates of the left corner of the object in px.
    :object_size: size of the object in pixels in CV2 coordinates (columns, rows)
    """
    frame_size = frame.shape
    print(left_corner)
    ROI = [[left_corner[0], left_corner[0] + object_size[0]],[left_corner[1], left_corner[1] + object_size[1]]]
    
    if left_corner[0] + object_size[0] > frame_size[0]:
        # outside the right bounding edge
        ROI[0] = [left_corner[0], frame_size[0]]
    if left_corner[0] < 0:
        # outside the left bounding edge
        ROI[0] = [0, left_corner[0] + object_size[0]]

    if left_corner[1] + object_size[1] > frame_size[1]:
        # outside the bottom bounding edge
        ROI[1] = [left_corner[1], frame_size[1]]
    if left_corner[1] < 0:
        # outside the upper bounding edge
        ROI[1] = [0, left_corner[1] + object_size[1]]

    return frame[ROI[0][0]:ROI[0][1], ROI[1][0]:ROI[1][1]]


def obj_in_frame(left_corner, frame, img):
    image_size = img.shape
    frame_size = frame.shape
    ROI = [[0,image_size[0]],[0,image_size[1]]]  # the whole image of the object
    if left_corner[0] + image_size[0] > frame_size[0]:
        # outside the right bounding edge
        ROI[0] = [0, frame_size[0] - left_corner[0]]
    if left_corner[0] < 0:
        # outside the left bounding edge
        ROI[0] = [-left_corner[0], image_size[0]]

    if left_corner[1] + image_size[1] > frame_size[1]:
        # outside the bottom bounding edge
        ROI[1] = [0, frame_size[1] - left_corner[1]]
    if left_corner[1] < 0:
        # outside the upper bounding edge
        ROI[1] = [-left_corner[1], image_size[1]]

    return img[ROI[0][0]:ROI[0][1], ROI[1][0]:ROI[1][1]] 

In [4]:
def create_mask(img):
    return np.zeros(img.shape[:2], dtype="uint8")

Orbital Mechanics

In [5]:
def propagate_based_on_vector(r,v, t):
    rf, vf = pykep.propagate_lagrangian(r0 = r, v0 = v, tof = t,mu = 398600)
    return np.array(rf), np.array(vf)

def propagate_based_on_elements(elements, t, kepler=True):
    r, v = pykep.par2ic(elements, mu=398600)
    rf,vf = pykep.propagate_lagrangian(r0 = r, v0 = v, tof = t,mu = 398600)
    if kepler == False:  # return vectors
        return np.array(rf),np.array(vf)
    if kepler == True:  # return kepler elements
        return pykep.ic2par(rf,vf,mu = 398600)

Utility

In [6]:
def delete_white_background(img, write=False):
    """
    Takes the images of Debris that I made in paint.
    Inverts the colours and deletes the white background.
    """
    img = np.invert(img)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    _, alpha = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY)
    
    # Using cv2.split() to split channels 
    # of coloured image
    b, g, r = cv2.split(img)

    # Making list of Red, Green, Blue
    # Channels and alpha
    rgba = [b,g,r, alpha]

    # Using cv2.merge() to merge rgba
    # into a coloured/multi-channeled image
    dst = cv2.merge(rgba,3)

    # Writing and saving to a new image
    if write is True:
        cv2.imwrite("debris/deb.png", dst)
    return dst        

FOV

In [176]:
# Checks if a matrix is a valid rotation matrix.
def isRotationMatrix(R) :
    Rt = np.transpose(R)
    shouldBeIdentity = np.dot(Rt, R)
    I = np.identity(3, dtype = R.dtype)
    n = np.linalg.norm(I - shouldBeIdentity)
    return n < 1e-6
 
# Calculates rotation matrix to euler angles
# The result is the same as MATLAB except the order
# of the euler angles ( x and z are swapped ).
def rotationMatrixToEulerAngles(R) :
 
    assert(isRotationMatrix(R))
 
    sy = math.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0])
 
    singular = sy < 1e-6
 
    if  not singular :
        x = math.atan2(R[2,1] , R[2,2])
        y = math.atan2(-R[2,0], sy)
        z = math.atan2(R[1,0], R[0,0])
    else :
        x = math.atan2(-R[1,2], R[1,1])
        y = math.atan2(-R[2,0], sy)
        z = 0
 
    return np.array([x, y, z])

In [177]:
def to_body_axis(vector, r, v):
    """
    Calculates the rotation matrix and applies it. Returns the converted vector.
    r and v are used to calculate the rotation matrix and vector is the vector to be 
    rotated.
    """
    
    unit_r = r/np.linalg.norm(r)
    unit_v = v/np.linalg.norm(v)

    R = np.array([[unit_v[0], -unit_r[0], unit_r[1]*unit_v[2] - unit_v[1]*unit_r[2]],
                [unit_v[1], -unit_r[1], unit_v[0]*unit_r[2] - unit_r[0]*unit_v[2]],
                [unit_v[2], -unit_r[2], unit_r[0]*unit_v[1] - unit_v[0]*unit_r[1]]])
    R = R.transpose()
    
    return R.dot(vector)

In [178]:
vec = [1,0,0]
r = [1,1,0]
v = [0,0,1]
to_body_axis(vec, r, v)

array([ 0.        , -0.70710678,  0.70710678])

In [194]:
def angle_between(r1,r2):
    """
    Calculate the angle between two vectors
    """
    unit_1 = r1/np.linalg.norm(r1)
    unit_2 = r2/np.linalg.norm(r2)
    return np.arccos(np.dot(unit_1,unit_2))

def behind_earth(r1, r2, R=6371):
    """
    Checks if the target is between the Earth and observer based on 
    a line segment intersecting the sphere.
    Returns true if the target is behind the Earth.
    :param r1: observer
    :param r2: target
    """
    if angle_between(r1,r2) < np.pi/2:  
        return False  # does not intersect
    
    phi = np.arccos(np.dot(r1, r2-r1)/(np.linalg.norm(r1)*np.linalg.norm(r2-r1)))
    H = np.linalg.norm(r1)*np.sin(phi)
    
    if H <= R:
        return True  # intersects
    if H > R:
        return False  # does not intersect

def in_FOV(r1, v1, r2, max_vertical=np.radians(60), max_horisontal=np.radians(60)):
    """
    Checks if the observer should be able to see the target.
    1 for observer. 2 for target.
    Returns True if yes
    """

    rel = r2 - r1
    
    if behind_earth(r1,r2) is True:
        print("failed cause earth")
        return False
    
    rel_unit = (rel)/np.linalg.norm(rel)
    print("relative in global", rel_unit)
    rel_unit = to_body_axis(rel_unit, r1, v1) 
    print("relative in body", rel_unit)
    if rel_unit[2] < 0:
        print("failed cause z")
        return False

    horisontal_angle = abs(np.arctan(rel_unit[0]/rel_unit[2]))  # in the xz plane
    vertical_angle = abs(np.arctan(rel_unit[1]/rel_unit[2]))  # in yz plane
    print("angles", np.degrees(horisontal_angle),np.degrees(vertical_angle))
    if horisontal_angle <= max_horisontal and vertical_angle <= max_vertical:
        return True
    else:
        print("failed cause angles")
        return False

In [188]:
r1 = np.array([7300,0,0])
r2 = np.array([7200,0,0])
behind_earth(r1, r2)

False

In [9]:
def get_distance(r1, v1, r2):
    rel = r2 - r1
    rel = to_body_axis(rel, r1, v1)
    distance = rel[2]
    return distance

def to_pixels(r,distance, cols=1980, rows=1080, alpha=np.pi / 6, beta=np.pi / 6,):
    """
    Converts physical location to pixels.
    """
    len_x = 2*np.tan(alpha)*distance
    len_y = 2*np.tan(beta)*distance
    px_per_length_cols =  cols/len_x
    px_per_length_rows =  rows/len_y
    return r*px_per_length_cols, r*px_per_length_rows


Initialisation

In [189]:
elements_sat = [7290.2,0.0610,np.radians(30.379),np.radians(289.042),np.radians(293.776),np.radians(66.230)]
elements_deb = [7133.8,0.0043,np.radians(98.289),np.radians(49.866),np.radians(356.768),np.radians(149.512)]

In [190]:
vs = "Earth\sized_Earth_slowed.avi"
img_path = "debris\deb1.png"

In [191]:
object_og = cv2.imread(img_path)
object_og = delete_white_background(object_og, write=False)

In [192]:
cap = cv2.VideoCapture(vs)  # open the video file
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out = cv2.VideoWriter('Earth_ball.avi', cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 30, (frame_width, frame_height))
t = 0
if cap.isOpened() is False:
    print("Error opening video file")

Body

In [212]:
while cap.isOpened():
    ret0, frame = cap.read()
    if ret0 is True:
        rO, vO = propagate_based_on_elements(elements_sat, t, kepler=False)
        rT, vT = propagate_based_on_elements(elements_deb, t, kepler=False)
        t+=1  # move ahead by one second
        if in_FOV(rO, vO, rT) is True:
            distance = get_distance(rO,vO,rT)
            object_new = resize(object_og, (1,1), distance)  # size of target in the frame
            if object_new is not False:
                left_corner = to_pixels(rT, distance)  # location of target in the frame
                roi = roi_in_frame(left_corner, object_new.size, frame)
                # create mask and inverse mask
                mask = create_mask(obj_in_frame(left_corner,frame,object_new))
                mask_inv = cv2.bitwise_not(mask)

                frame_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)
                object_fg = cv2.bitwise_and(object_new,object_new, mask=mask)

                dst = cv2.add(frame_bg, object_fg)
                frame[left_corner[0]:roi.shape[0]+left_corner[0],left_corner[1]:roi.shape[1]+left_corner[1]]
                
                cv2.imshow("Earth with ball", frame)
                cv2.waitKey(0)
        out.write(frame)
    else:
        break

out.release()
cap.release()
cv2.destroyAllWindows()

relative in global [-0.83826581  0.22320087  0.49748548]
relative in body [-0.36476545  0.50749595  0.79183493]
angles 24.73356033281702 32.65626925255723
5788.988920655761
lengths 6684.548623352729 6684.548623352729
(array([-1017.28573049, -1465.79396677,  1146.71679859]), array([-554.88312572, -799.52398188,  625.48189014]))


TypeError: 'int' object is not subscriptable